# Stem Exploration

This notebook provides code to analyse and explore point cloud tree stems. As input you can either use an already separated stem point cloud or use a complete tree and use the provided separation code (see `option B` in step 1)

---------------

##### Imports

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import os
import glob
from tqdm import tqdm
from pathlib import Path
import pandas as pd
import trimesh
import numpy as np
import open3d as o3d
import logging as log
import tree as tree_utils
import utils.o3d_utils as o3d_utils
import utils.plot_utils as plot_utils
import utils.ahn_utils as ahn_utils
import utils.las_utils as las_utils
import utils.math_utils as math_utils
from misc.fitcyclinders import fit_vertical_cylinder_3D
from utils.interpolation import FastGridInterpolator

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

from labels import Labels

### 1. Load Data
---

**Option A**: Load stem point cloud

In [ ]:
treecode = '119305_485108'
# las_file = '../datasets/sonarski/sonarski_'+treecode+'.las'
# las_file = '../datasets/ahn/ahn_'+treecode+'.las'
las_file = '../../../datasets/clipped_trees/clipped_trees_ahn/single/single_'+treecode+'.las'

tree_cloud = o3d_utils.read_las(las_file)

voxel_size = 0.05
tree_cloud_voxeled = tree_cloud.voxel_down_sample(voxel_size)

In [ ]:
o3d.visualization.draw_geometries([tree_cloud_voxeled])

### 2. Stem Analysis
---

In [ ]:
# def crown_base_height(pcd, z_min=None, z_step=.1, th=.3):

#     # Vertical density
#     z = np.asarray(pcd.points)[:,2]
#     z_bins = np.arange(z.min(), z.max()+z_step, z_step)
#     hist, bin_edges = np.histogram(z, z_bins)

#     # Find inflection (smooth)
#     smooth = gaussian_filter1d(hist, 2)
#     smooth[smooth == 0] = 1
#     change = (smooth[1:]-smooth[:-1]) / smooth[:-1]
#     z_ind = np.argmax(change > th) + 1
#     # plt.barh(np.arange(len(smooth)), smooth, height=.8, alpha=.5)
#     # plt.hlines(z_ind, 0, np.max(hist), linestyles='--', color='r')

#     z_split = bin_edges[z_ind]+z_step/2

#     return z_split

def has_stem(pcd, z_min=0, z_step=.1, min_points=20, th=.8):
    z = np.asarray(pcd.points)[:,2]
    z_max = z_min + 2
    z = z[z<z_max]
    z_bins = np.arange(z_min, z_max+z_step, z_step)
    hist, bin_edges = np.histogram(z, z_bins)
    if len(hist) > 0:
        d = hist > min_points
        return np.sum(d) / len(d) > th
    else:
        return False

def get_all_stem_infos(datasets=['ahn','cyclomedia','sonarski']):
    records = []

    data_root = Path('../../../datasets/clipped_trees/')
    file_types = ('.LAS', '.las', '.LAZ', '.laz')

    # parameters
    voxel_size = 0.05

    for dataset in datasets:
        in_folder = data_root.joinpath('clipped_trees_'+dataset+'/single/')

        files = []

        for f in in_folder.glob('*'):
            if f.name.endswith(file_types):
                files.append(f)

        for file in tqdm(files):
            treecode = las_utils.get_treecode_from_filename(file.name)
            
            # load las
            tree_cloud = o3d_utils.read_las(file)
            tree_cloud_voxeled = tree_cloud.voxel_down_sample(voxel_size)

            z_split = crown_base_height(tree_cloud_voxeled)
            stem = has_stem(tree_cloud_voxeled, z_split)

            records.append({'treecode':treecode,'data':dataset,'split':z_split,'stem':stem})

    return pd.DataFrame(records)


In [ ]:
df = get_all_stem_infos()

In [ ]:
df[df['split']>1]

In [ ]:
treecode = '121913_487434'
las_file = '../datasets/ahn/ahn_'+treecode+'.las'
treecode = las_utils.get_treecode_from_filename(las_file)
pcd = o3d_utils.read_las(las_file)

ahn_data_folder = '../datasets/ahn_surf/'
npz_reader = ahn_utils.NPZReader(ahn_data_folder)
tree_z_min = npz_reader.tree_surface_z(treecode)[0]

### Stem vis
---

In [ ]:
from misc.fitcyclinders import fit_vertical_cylinder_3D

ahn_data_folder = '../datasets/waternet_set/ahn_surf/'
npz_reader = ahn_utils.NPZReader(ahn_data_folder)

In [ ]:
treecode = '119657_484841'
tree_min_z = npz_reader.tree_surface_z(treecode)

las_file = '../datasets/waternet_set/clipped/cyclo_119667_484841_Tilia_4.las'

# sonarski_cloud = o3d_utils.read_las('../datasets/sonarski/sonarski_'+treecode+'.las')
cyclo_cloud = o3d_utils.read_las(las_file)

In [ ]:
def slice_pcd(pcd, z, height):
    points = np.array(pcd.points)
    mask = (points[:,2] >= z - height/2) & (points[:,2] <= z + height/2)
    slice_pcd = pcd.select_by_index(np.where(mask)[0])
    return slice_pcd

In [ ]:
def plot_stem_slice(pcd, z, width, center):

    sample = slice_pcd(pcd, z, width)

    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(projection='3d')
    points = np.array(sample.points)
    points -= center
    xs, ys, zs = points.T
    colors = np.asarray(sample.colors)
    img = ax.scatter(xs, ys, zs, s=3, c=colors)

    ax.set_xlabel('Meter')
    ax.set_ylabel('Meter')
    ax.set_zlabel('Meter')
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    ax.set_zlim(-1,1)
    ax.view_init(25,-65)
    plt.savefig('stem_slice.png', transparent=True)
    plt.show()

In [ ]:
pcd_ = slice_pcd(cyclo_cloud, tree_min_z+1.3, .4)
center_ = pcd_.get_center()

In [ ]:
plot_stem_slice(cyclo_cloud, tree_min_z+1.3, .4, center_)

In [ ]:
sample = slice_pcd(cld, tree_min_z+1.3, .4)

points = np.array(sample.points)
points -= np.mean(points, axis=0)
xs, ys, zs = points.T
colors = np.asarray(sample.colors)

# fit circle
center, _, radius, inliers, cci = fit_vertical_cylinder_3D(points, 0.02)


In [ ]:
fig, (ax) = plt.subplots(1, 1, figsize=(9,6))
img = ax.scatter(xs, ys, s=2, c=colors, marker='.')
Drawing_uncolored_circle = plt.Circle( center[:2],
                                    radius ,
                                    fill = False ,
                                    color= 'red')
ax.add_artist( Drawing_uncolored_circle )
ax.scatter(*center[:2], marker='x', s=10, color='red')
ax.plot([center[0],center[0]+radius],[center[1],center[1]], c='red', alpha=0.6, linestyle='--')
ax.text(center[0]+0.05,center[1]+.02, 'r = '+str(np.round(radius,3)))

ax.set_xlabel('Meter')
ax.set_xlim(-.7,.7)
ax.set_ylim(-.7,.7)
ax.set_aspect('equal')
ax.set_title('DBH')

In [ ]:
from scipy.optimize import leastsq
p = [0, 0, 0, 0, max(np.abs(ys).max(), np.abs(xs).max())/2]

# fit
fitfunc = lambda p, x, y, z: (- np.cos(p[3])*(p[0] - x) - z*np.cos(p[2])*np.sin(p[3]) - np.sin(p[2])*np.sin(p[3])*(p[1] - y))**2 + (z*np.sin(p[2]) - np.cos(p[2])*(p[1] - y))**2 #fit function
errfunc = lambda p, x, y, z: fitfunc(p, x, y, z) - p[4]**2 #error function 
est_p = leastsq(errfunc, p, args=(xs, ys, zs), maxfev=1000)[0]
inliers = np.where(np.abs(errfunc(est_p,xs,ys,zs))<.02)[0]

In [ ]:
from pyransac3d import Circle

In [ ]:
from scipy.spatial.transform import Rotation as R

center = np.array([est_p[0],est_p[1],0]) + center_
radius = est_p[4]

rotation = R.from_rotvec([est_p[2], 0, 0])
axis = rotation.apply([0,0,1])
rotation = R.from_rotvec([0, est_p[3], 0])
axis = rotation.apply(axis)

# circumferential completeness index (CCI)
P_xy = math_utils.rodrigues_rot(points, axis, [0, 0, 1])
CCI = math_utils.circumferential_completeness_index([est_p[0], est_p[1]], radius, P_xy)

mesh = trimesh.creation.cylinder(radius=radius,
                 sections=25, 
                 segment=(center+axis*zs.min(),center+axis*zs.max())).as_open3d
mesh_lines = o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
mesh_lines.paint_uniform_color((0, 0, 0))

inliers_pcd = sample.select_by_index(inliers)
outlier_pcd = sample.select_by_index(inliers, invert=True)
outlier_pcd.paint_uniform_color([1,0,0])

o3d.visualization.draw_geometries([inliers_pcd, outlier_pcd, mesh_lines])

In [ ]:
sample = slice_pcd(cyclo_cloud, tree_min_z+1.3, 2.6)
cld, ind = sample.remove_statistical_outlier(16, 1.0)
inlier_cloud = sample.select_by_index(ind)
outlier_cloud = sample.select_by_index(ind, invert=True)
outlier_cloud.paint_uniform_color([1,0,0])
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [ ]:
tree_min_z = cyclo_cloud.get_min_bound()[2]